# Obtén datos de la API de Marvel

![imagen](https://cdn.hobbyconsolas.com/sites/navi.axelspringer.es/public/styles/hc_1440x810/public/media/image/2021/09/marvel-2460339.jpg?itok=cBnC1CBi)

Te tendrás que [registrar](https://developer.marvel.com/) y consultar la [documentación](https://developer.marvel.com/docs)

Queremos que consultes a la api para que te devuelva la info de los personajes de marvel que empiecen por la inicial de tu nombre. Deberemos guardarlos en un csv la información con la siguiente estructura

![imagen](./img/Captura_marvel.PNG)

Aquí te dejamos el código en python para poder empezar a hacer las llamadas, rellenando algunas variables como tus keys, parámetros de la llamada y la url (endpoint) a la que quieres acceder

In [15]:
import hashlib
import requests
import datetime
import pandas as pd

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

In [16]:
def hash_params(timestamp,priv_key,pub_key):
    """ Marvel API requires server side API calls to include
    md5 hash of timestamp + public key + private key """

    hash_md5 = hashlib.md5()
    hash_md5.update(f'{timestamp}{priv_key}{pub_key}'.encode('utf-8'))
    hashed_params = hash_md5.hexdigest()

    return hashed_params

In [17]:
def get_marvel_characters(initial, max_characters=1000):

    timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')

    pub_key = '7b519eca220cea84b556d6da07c75bbe'
    priv_key = 'd097b014e0a40e56706de63cd61b3a954601a1de'

    url = 'http://gateway.marvel.com/v1/public/characters'

    all_characters = []
    offset = 0
    limit = 100

    while offset < max_characters:
        params = {
            'ts': timestamp,
            'apikey': pub_key,
            'hash': hash_params(timestamp, priv_key, pub_key),
            'nameStartsWith': initial,
            'offset': offset,
            'limit': limit
        }

        res = requests.get(url, params=params)
        
        if res.status_code == 200:
            data = res.json()
            characters = data['data']['results']
            all_characters.extend(characters)
            
            total = data['data']['total']
            offset += limit
            
            if offset >= total:
                break
        else:
            print(f"Error en la solicitud. Código de estado: {res.status_code}")
            break

    df = pd.DataFrame([
        {
            'id': char['id'],
            'name': char['name'],
            'picture_url': f"{char['thumbnail']['path']}.{char['thumbnail']['extension']}"
        }
        for char in all_characters
    ])

    return df

initial = "M"  # O cualquier otra inicial
df_marvel = get_marvel_characters(initial, max_characters=1000)
df_marvel


,id,name,picture_url
0,1011068,M (Monet St. Croix),http://i.annihil.us/u/prod/marvel/i/mg/b/80/4c0030eabc66f.jpg
1,1011004,M.O.D.A.M.,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
2,1011320,M.O.D.O.G.,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
3,1010726,M.O.D.O.K.,http://i.annihil.us/u/prod/marvel/i/mg/0/03/526034ba37851.jpg
4,1017321,M.O.D.O.K. (Iron Man 3 - The Official Game),http://i.annihil.us/u/prod/marvel/i/mg/c/00/5239c14fcddc0.jpg
...,...,...,...
152,1010952,Mysterio (Francis Klum),http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
153,1009465,Mystique,http://i.annihil.us/u/prod/marvel/i/mg/5/03/5390dc2225782.jpg
154,1010953,Mystique (Age of Apocalypse),http://i.annihil.us/u/prod/marvel/i/mg/4/03/528d33b473550.jpg
155,1010954,Mystique (House of M),http://i.annihil.us/u/prod/marvel/i/mg/b/70/4c0033fbbdb54.jpg


In [18]:
df_marvel.to_csv("data/marvelsuperheroestrueall.csv", sep = ';', index = True)